# Quickstart Guide 
この Notebook は、LangChain の Quickstart Guide を以下の条件で書き換えたものになります。
- Azure OpenAI Service を使用
- モデルは
    - text-davinchi-003 (デプロイ名：text-davinchi-003)
    - gpt-4 (デプロイ名：gpt-4)

以下のリンクは、オリジナルのサイトになります。こちらも参照するようにしてください。  
https://python.langchain.com/en/latest/getting_started/getting_started.html

### Enironment Setup
必要なパラメータの値を、.env ファイルより取得します。あらかじめ.env ファイルに必要な値を設定しておいてください。

In [ ]:
import os
from dotenv import load_dotenv
import openai

# .env ファイルから環境変数を読み込む
load_dotenv()

# モデルのデプロイ名をパラメータに設定
AZURE_OPENAI_GPT_DEPLOYMENT = os.environ.get("AZURE_OPENAI_GPT_DEPLOYMENT")    # text-davinchi-003 のデプロイ名
AZURE_OPENAI_GPT4_DEPLOYMENT = os.environ.get("AZURE_OPENAI_GPT4_DEPLOYMENT")  # gpt-4 のデプロイ名

# OpenAI API が使用するパラメータの設定
openai.api_type = "azure"
openai.api_base = os.environ.get("OPENAI_API_BASE")                            # Azure OpenAI API エンドポイント
openai.api_version = os.environ.get("OPENAI_API_VERSION")                      # Azure OpenAI API バージョン
openai.api_key = os.environ.get("OPENAI_API_KEY")                              # Azure OpenAI API キー

### Building a Language Model Application: LLMs
LangChainには、言語モデルアプリケーションを構築するために使用できる多数のモジュールがあります。モジュールを組み合わせてより複雑なアプリケーションを作成することもでき、単純なアプリケーションに個別に使用することもできます。

### LLMs: Get predictions from a language 
LangChain の最も基本的な構成要素は、入力に対して LLM を呼び出すことです。これを行う簡単な例を紹介しましょう。ここでは企業が製造している商品に基づいて会社名を生成するサービスを構築するとします。  
これを行うためには、まず LLM ラッパーをインポートする必要があります。

In [ ]:
from langchain.llms.openai import AzureOpenAI

その後、任意の引数でラッパーを初期化することができます。この例では、text-davinchi-003 モデルを使用し、出力をよりランダムにするための高めの temperature で初期化します。

In [ ]:
llm = AzureOpenAI(deployment_name=AZURE_OPENAI_GPT_DEPLOYMENT, temperature=0.9, openai_api_key=openai.api_key)

それでは、何かしらの入力に対してLLMを呼び出してみましょう！

In [ ]:
text = "What would be a good company name for a company that makes colorful socks?"
print(llm(text))

### Prompt Templates: Manage prompts for LLMs
LLM を呼び出すことは素晴らしい最初のステップになりますが、それは始まりにすぎません。通常、アプリケーションで LLM を使用する場合、ユーザーの入力を直接 LLM に送信するわけではありません。代わりに、ユーザーの入力を取得し、プロンプトを構築してから LLMに送信することが多いでしょう。  
例えば前の例では、私たちが渡したテキストはカラフルな靴下を製造する会社の名前を求めるようにハードコードされていました。この想像上のサービスでは、ユーザーが会社が何をするかを説明する入力のみを受け取り、その情報を持ったプロンプトをフォーマットしたいと思うでしょう。  
これはLangChainで簡単に行えます！  
まず、プロンプトのテンプレートを定義しましょう。

In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["product"],
    template="What would be good company name for a company that makes {product}?"
    )

それでは、これがどのように機能するか見てみましょう！ `.format` メソッドを呼び出して、プロンプトをフォーマットします。

In [ ]:
print(prompt.format(product="colorful socks"))

### Chains: Combine LLMs and prompts in multi-step workflows
ここまで、PromptTemplate や LLM といったプリミティブを単独で扱ってきましたが、実際のアプリケーションは単一のプリミティブではなく、それらを組み合わせたものです。
LangChain のチェーンは、LLM や他のチェーンなどのプリミティブとなるリンクで構成されています。
最も基本的なチェーンのタイプは、PromptTemplate と LLM から構成される LLMChain です。
前の例を拡張して、ユーザーの入力を取得し、PromptTemplate でフォーマットし、フォーマットされた応答を LLM に渡す LLMChain を構築します。  

In [ ]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)

それでは、製品を指定するだけのチェーンを実行します！

In [ ]:
chain.run("color socks")

やりましたね！ これが最初のチェーン、LLM Chain です。これはよりシンプルなチェーンのタイプの1つですが、その動作原理を理解することは、より複雑なチェーンを扱うための良いスタートになります。

### Agents: Dynamically Call Chains Based on User Input
今まで見てきたチェーンは、あらかじめ決められた順序で実行されていました。

しかし、エージェントは異なります。エージェントは LLM を使用して、どのアクションを実行するか、そしてどの順序で実行するかを決定します。アクションは、ツールを使用して出力を監視する、もしくはユーザーに返すことができます。

エージェントを正しく使用すると、非常に強力なものになります。このチュートリアルでは、最もシンプルでハイレベルの API を通じて、エージェントを簡単に使用する方法を紹介します。

エージェントをロードするには、以下の概念を理解する必要があります。
- ツール: 特定の役割を果たす関数です。これには、Google 検索、データベースの検索、Python REPL、他のチェーンなどが含まれます。ツールのインターフェイスは、現在、文字列を入力として受け取り、文字列を出力として返す関数である必要があります。
- LLM: エージェントの駆動力となる言語モデル。
- エージェント: 使用するエージェントです。これは、サポートされているエージェントクラスを参照する文字列である必要があります。このノートブックでは、最もシンプルでハイレベルのAPIに焦点を当てているため、標準のサポートされているエージェントのみを使用する方法について説明しています。

ここでは SerpAPI を使用します。SerpAPI は、サーチエンジンをスクレイプする API で、以下のサイトでサインアップして API Key を取得する必要があります。  
https://serpapi.com/  

API Key の値を環境変数として設定する必要がありますが、これは.env ファイルからすでに読み込み済みです。  

それでは始めてみましょう。

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent

# text-davinchi-003 モデルをロード
# ここでは Temperature を最も低い値に設定
llm = AzureOpenAI(deployment_name=AZURE_OPENAI_GPT_DEPLOYMENT, temperature=0, openai_api_key=openai.api_key)

# 使用するツールをロード
# llm-math は数値計算のために使用
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# 使用するツール、言語モデル、そして使用するエージェントの種類を指定して、エージェントを初期化
agent = initialize_agent(tools=tools, llm=llm, agent="zero-shot-react-description", verbose=True)

# エージェントの実行
agent.run("What was the high temperature in SF yesterday in Fahrenheit? What is that number raised to the .023 power?")

### Memory: Add State to Chains and Agents
これまでに私たちが経験してきたすべてのチェーンやエージェントはステートレスでした。しかし、時にはチェーンやエージェントが前回のやりとりに関する情報を記憶できるようにするために、「メモリ」の概念を持つことが望ましい場合があります。最も明確で単純な例は、チャットボットを設計するときです。チャットボットが前のメッセージを記憶して、それを利用してより良い会話を行えるようにすることが望まれます。これは「短期記憶」の一種です。より複雑な例では、チェーン/エージェントが時間の経過とともに重要な情報を記憶することができるようにすることができます。これは「長期記憶」の一形態になります。

LangChain には、この目的のために特別に作成されたいくつかのチェーンがあります。このノートブックでは、このようなチェーンの1つである、2種類のメモリを持つ`ConversationChain` を使用する方法について説明します。

デフォルトでは、`ConversationChain` は、以前のすべての入力/出力を記憶してコンテキストに追加する単純なタイプのメモリを持ちます。このチェーンを使用する方法を見てみましょう（プロンプトを表示するために `verbose=True` を設定します）。

In [ ]:
from langchain import ConversationChain

conversation = ConversationChain(llm=llm, verbose=True)

output = conversation.predict(input="Hi there!")
print(output)

In [ ]:
output = conversation.predict(input="I'm doing well!  Just having a conversation with an AI.")

print(output)

### Building a Language Model Application: Chat Models
同様に、LLM の代わりにチャットモデルを使用することができます。チャットモデルは、言語モデルの1つのバリエーションです。チャットモデルは内部で言語モデルを使用している一方で、公開されているインターフェースは少し異なります。 「テキストを入力し、テキストを出力する」API ではなく、「チャットメッセージ」が入力と出力の両方になるインターフェースを公開しています。  
チャットモデル API は比較的新しいため、正しい抽象化方法をまだ見つけている途中です。

### Get Message Completions from a Chat Model
チャットモデルに1つまたは複数のメッセージを渡すことで、チャットの Completion を取得できます。応答はメッセージになります。現在、LangChain でサポートされているメッセージの種類には、`AIMessage`、`HumanMessage`、`SystemMessage`、および`ChatMessage` があります。`ChatMessage` は任意の役割パラメータを受け取ります。ほとんどの場合、`HumanMessage`、`AIMessage`、および `SystemMessage` を扱うことになるでしょう。

In [ ]:
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import (
    HumanMessage, 
    SystemMessage
)

chat = AzureChatOpenAI(deployment_name=AZURE_OPENAI_GPT4_DEPLOYMENT, temperature=0, openai_api_key=openai.api_key)

単一のメッセージを渡すことで、Completion を取得することができます。

In [ ]:
chat([HumanMessage(content="Translate this sentence from English to Japanese. I love programming.")])

OpenAI の gpt-3.5-turbo と gpt-4 モデルでは、複数のメッセージを渡すこともできます。

In [ ]:
message = [SystemMessage(content="You are a helpful assistant that translate English to Japanese."),
           HumanMessage(content="Translate this sentence from English to Japanese. I love programming.")]

chat(message)

さらに、`generate` を使用して複数のメッセージセットに対して Completion を生成することもできます。この場合は、追加の `message` パラメータを持つ`LLMResult` が返されます。

In [ ]:
batch_messages = [
    [
        SystemMessage(content="You are a helpful assistant that translates English to Japanese."),
        HumanMessage(content="Translate this sentence from English to Japanese. I love programming.")
    ],
    [
        SystemMessage(content="You are a helpful assistant that translates English to Japanese."),
        HumanMessage(content="Translate this sentence from English to Japanese. I love artificial intelligence.")
    ],
]

result = chat.generate(batch_messages)
print(result)

この LLMResult からトークンの使用状況などを取得することができます。

In [ ]:
result.llm_output['token_usage']

### Chat Prompt Templates
LLM と同様に、`MessagePromptTemplate` を使用したテンプレートを利用することができます。`MessagePromptTemplate` から `ChatPromptTemplate` を構築することができ、`ChatPromptTemplate` では `format_prompt` を使用することができます。これは `PromptValue` を返しますが、フォーマットされた値を LLM またはチャットモデルの入力として使用するかどうかに応じて、文字列またはメッセージオブジェクトに変換することができます。

テンプレートに対して使用できる `from_template` メソッドがあります。このテンプレートを使用すると、以下のように記述することができます。

In [ ]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chat(chat_prompt.format_prompt(input_language="English", output_language="Japanese", 
                               text="I love programming.").to_messages())


### Chains with Chat Models
前述のセクションで説明した `LLMChain` は、チャットモデルでも使用することができます。

In [ ]:
template = "You ara a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chain = LLMChain(llm=chat, prompt=chat_prompt)
chain.run(input_language="English", output_language="Japanese", text="I love programming.")

### Agents with Chat Models
エージェントは、チャットモデルでも使用できます。エージェントを初期化する場合は、エージェントタイプとして「zero-shot-react-description」を使用することができます。

In [ ]:
# 使用するツールをロード
# llm-math は数値計算のために使用
tools = load_tools(["serpapi", "llm-math"], llm=chat)

# 使用するツール、言語モデル、そして使用するエージェントの種類を指定して、エージェントを初期化
agent = initialize_agent(tools=tools, llm=chat, agent="zero-shot-react-description", verbose=True)

# エージェントの実行
agent.run("Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?")

### Memory: Add State to Chains and Agents
チャットモデルを使用して初期化されたチェーンとエージェントで Memory を使用することができます。LLM の Memory との主な違いは、過去のすべてのメッセージを文字列にまとめるのではなく、独自のユニークなメモリオブジェクトとして保持できることです。

In [ ]:
from langchain.prompts import MessagesPlaceholder
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

llm = AzureChatOpenAI(deployment_name=AZURE_OPENAI_GPT4_DEPLOYMENT, temperature=0, openai_api_key=openai.api_key)
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm) 

conversation.predict(input="Hi there!")

conversation.predict(input="I'm doing well! Just having a conversation with an AI.")

conversation.predict(input="Tell me about yourself.")

Memory に格納されたメッセージを確認してみましょう。

In [ ]:
memory.load_memory_variables({})

要求プロンプトに入力できるトークン数には制限があるため、必要に応じて格納されたメッセージを消去します。

In [ ]:
memory.clear()